# Detect pipeline
This notebook will run through how to use the detect function on a subset of the Tasman Sea domain. 

I'm primarily writing this as the PBS job has done some funny things and i'm hoping this will catch out whatever is tripping that up. 

In [1]:
import xarray as xr
import numpy as np
from timeit import default_timer as timer
from datetime import timedelta
import glob
from xmhw.xmhw import threshold
import os

In [2]:
from dask.distributed import Client
client = Client()
client

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46455 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/46455/status,
Dashboard: /proxy/46455/status,Workers: 4
Total threads: 8,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37201,Workers: 4
Dashboard: /proxy/46455/status,Total threads: 8
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:36285,Total threads: 2
Dashboard: /proxy/44787/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:39641,


In [ ]:
import mhw_scripts

In [3]:
def open_files_after_date(path, pattern, startyear):
    ds_list = glob.glob(path + pattern)
    ds_list.sort()
    newlist = []
    i = 0
    while f'{startyear}' not in ds_list[i]:
        i+=1
    first = i
    newlist = ds_list[first:]
    return newlist

### Data 

In [4]:
# EAC-003 paths
# eac-zarr
zarr_path = '/home/581/jr5971/scratch/mom6-setonix/subsetted_data/zarr-files'
east_tas_eac003_path = zarr_path + '/eac-thetao-time_rc.zarr'

In [6]:
east_tas_eac003 = xr.open_zarr(east_tas_eac003_path, consolidated=True)['thetao']
east_tas_eac003

<xarray.DataArray 'thetao' (time: 3440, z_l: 100, yh: 162, xh: 90)>
dask.array<open_dataset-thetao, shape=(3440, 100, 162, 90), dtype=float32, chunksize=(3440, 18, 31, 16), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2008-01-01 12:00:00 ... 2017-12-31 12:00:00
  * xh       (xh) float64 -213.0 -212.9 -212.9 -212.9 ... -210.1 -210.0 -210.0
  * yh       (yh) float64 -43.99 -43.97 -43.94 -43.92 ... -40.05 -40.03 -40.0
  * z_l      (z_l) float64 0.2928 0.8999 1.552 ... 4.946e+03 5.134e+03 5.324e+03
Attributes:
    cell_measures:  volume: volcello area: areacello
    cell_methods:   area:mean z_l:mean yh:mean xh:mean time: mean
    long_name:      Sea Water Potential Temperature
    standard_name:  sea_water_potential_temperature
    time_avg_info:  average_T1,average_T2,average_DT
    units:          degC

In [ ]:
# open files after date
filelist = open_files_after_date(path='/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v140_iaf/output*/', 
                                 pattern = 'ocean-2d-surface_temp-1-daily-mean-ym_*.nc',
                                 startyear = '1990')


In [6]:
ds_list = glob.glob('/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_jra55v140_iaf/output*/ocean/ocean-2d-surface_temp-1-daily-mean-ym_*.nc')
ds_list.sort()
newlist = []
i = 0
while 'ym_1990' not in ds_list[i]:
    i+=1
first = i
newlist = ds_list[first:]

ds = xr.open_mfdataset(newlist, parallel=True)
sst = ds['surface_temp']

In [7]:
idx_reg = {'W':-217, 'E':-184,'S':-48,'N':-10}
sst = sst.sel(yt_ocean=slice(idx_reg['S'], idx_reg['N']),
              xt_ocean=slice(idx_reg['W'], idx_reg['E']))
lat_cellslice=[414, 440]
sst = sst.isel(yt_ocean=slice(lat_cellslice[0], lat_cellslice[1]))

In [12]:
sst

<xarray.DataArray 'surface_temp' (time: 10592, yt_ocean: 26, xt_ocean: 330)>
dask.array<sub, shape=(10592, 26, 330), dtype=float32, chunksize=(10592, 3, 330), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -216.9 -216.8 -216.7 ... -184.2 -184.1 -184.0
  * yt_ocean  (yt_ocean) float64 -13.33 -13.23 -13.13 ... -11.08 -10.98 -10.88
  * time      (time) datetime64[ns] 1990-01-01T12:00:00 ... 2018-12-31T12:00:00

In [9]:
# ny_blocks = 8
# y_chunks = len(sst.yt_ocean) // ny_blocks
# nx_blocks = 1
# x_chunks = len(sst.xt_ocean) // nx_blocks

# sst = sst.chunk({'time':-1, 'xt_ocean':x_chunks, 'yt_ocean':y_chunks})
# sst = sst - 273.15

In [8]:
# I want single depth levels...
temp.chunks

((3440,),
 (18, 18, 18, 18, 18, 10),
 (31, 31, 31, 31, 31, 7),
 (16, 16, 16, 16, 16, 10))

In [9]:
temp_rc = temp.chunk({'time':-1, 'xh':-1, 'yh':-1, 'z_l':1})
temp_rc

<xarray.DataArray 'thetao' (time: 3440, z_l: 100, yh: 162, xh: 90)>
dask.array<rechunk-merge, shape=(3440, 100, 162, 90), dtype=float32, chunksize=(3440, 1, 162, 90), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2008-01-01 12:00:00 ... 2017-12-31 12:00:00
  * xh       (xh) float64 -213.0 -212.9 -212.9 -212.9 ... -210.1 -210.0 -210.0
  * yh       (yh) float64 -43.99 -43.97 -43.94 -43.92 ... -40.05 -40.03 -40.0
  * z_l      (z_l) float64 0.2928 0.8999 1.552 ... 4.946e+03 5.134e+03 5.324e+03
Attributes:
    cell_measures:  volume: volcello area: areacello
    cell_methods:   area:mean z_l:mean yh:mean xh:mean time: mean
    long_name:      Sea Water Potential Temperature
    standard_name:  sea_water_potential_temperature
    time_avg_info:  average_T1,average_T2,average_DT
    units:          degC

In [7]:
temp = east_tas_eac003
lon = 'xh'
lat = 'yh'
z = 'z_l'
ny_blocks = 8
y_chunks = len(temp[lat]) // ny_blocks
nx_blocks = 1
x_chunks = len(temp[lon]) // nx_blocks
nz_blocks = 1
z_chunks = len(temp[z]) // nz_blocks
print(y_chunks, x_chunks, z_chunks)

20 90 100


In [10]:
_, yt, xt = sst.chunks
xtstep = xt[0]
ytstep = yt[0]
xtblocks = len(xt)
ytblocks = len(yt)

In [19]:
_, zt, yt, xt = temp_rc.chunks
ztstep = zt[0]
xtstep = xt[0]
ytstep = yt[0]
ztblocks = len(zt)
xtblocks = len(xt)
ytblocks = len(yt)

In [20]:
print(ztstep, xtstep, ytstep, ztblocks, xtblocks, ytblocks)

1 90 162 100 1 1


In [16]:
from xmhw.xmhw import detect

In [24]:
mhw_result_path = '/home/581/jr5971/jr5971/mhw-analysis/eac-003_easttas/'


In [28]:
temp['time'] = temp['time'].astype('datetime64[ns]')
temp['time']

<xarray.DataArray 'time' (time: 3440)>
array(['2008-01-01T12:00:00.000000000', '2008-01-02T12:00:00.000000000',
       '2008-01-03T12:00:00.000000000', ..., '2017-12-29T12:00:00.000000000',
       '2017-12-30T12:00:00.000000000', '2017-12-31T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2008-01-01T12:00:00 ... 2017-12-31T12:00:00
Attributes:
    axis:           T
    bounds:         time_bnds
    calendar_type:  JULIAN
    long_name:      time

In [ ]:
%%time
results_th = []
results_det = []

for z in range(ztblocks):
    start_loop = timer()
    ts = temp.isel(z_l=z).load()
    results_th.append(threshold(ts))
    print(f'Chunk {z} of size {ts.nbytes/1e6} MB is finished! Time: {str(timedelta(seconds=timer()-start_loop))}')
    results_th[z].to_netcdf(f"{mhw_result_path}/temp_res/res_{z}.nc")
    print('and saved!')
    
    print(f'Starting detect func {z}')
    se = results_th[z_level]['seas']
    th = results_th[z_level]['thresh']
    mhw, intermediate = detect(ts, th=th, se=se, intermediate=True)
    print(f'Finished detecting mhws on chunk {z+1}')
    mhw.to_netcdf(mhw_result_path + f'tmp_results/mhw_res_{z+1}.nc')
    intermediate.to_netcdf(mhw_result_path + f'tmp_results/int_res_{z+1}.nc')
    print('and saved mhw results')
    del(ts, ds_th, th, se, mhw, intermediate)
    z+=1

    

calendar not in keys


/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.10/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 6.46 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
%%time
results_th = []
results_det = []

for z_level in range(ztblocks):
    start_loop = timer()
    ts = temp.isel(z_l=z_level).load()
    results_th.append(threshold(ts))
    print(f'Chunk {z_level} of size {ts.nbytes/1e6} MB is finished! Time: {str(timedelta(seconds=timer()-start_loop))}')
    results_th[z_level].to_netcdf(f"{mhw_result_path}temp_res/res_{z_level}.nc")
    

for j in range(ytblocks):
    start_loop = timer()
    yt_from = j*ytstep
    yt_to = (j+1)*ytstep
    ts = sst.isel(yt_ocean=slice(yt_from, yt_to)).load()
    results_th.append(threshold(ts))

    print(f"Chunk {j+1} of size {ts.nbytes/1e6} MB is finished! Time: {str(timedelta(seconds=timer()-start_loop))}")
        
    results_th[j].to_netcdf(f"{mhw_result_path}temp_res/res_{j}.nc")
    print('and saved!')
    print(f'Starting detect func {j}')
    se = results_th[j]['seas']
    th = results_th[j]['thresh']
    results_det.append(detect(ts, th=th, se=se))
    results_det[j].to_netcdf(f'/g/data/v45/jr5971/mhw-analysis/mhw-scaling-tests/4xdata4/detect/chunks/res_{j}.nc')
    del(ts, th, se)
    j+=1
    